In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt
import os 
import re

In [2]:
df_regions=pd.read_csv("/neurospin/dico/fred/Runs/01_betaVAE/Program/2023_jlaval_STSbabies/betaVAE/notebooks/fred/PEPR_Marseille/Selected subjects/For each subject/Evaluation_metrics_with_v4/Quantile for each subject/List_of_regions")

In [3]:
df_regions

,Unnamed: 0,index_subject,list of regions
0,0,5,"[('S.F.median-S.F.pol.tr.-S.F.sup.', 'R'), ('S..."
1,1,6,"[('S.Call.-S.s.P.-S.intraCing.', 'L'), ('S.s.P..."
2,2,14,"[('S.T.i.-S.T.s.-S.T.pol.', 'L'), ('S.T.i.-S.O..."
3,3,16,"[('F.Coll.-S.Rh.', 'L'), ('S.T.i.-S.T.s.-S.T.p..."
4,4,17,"[('S.F.median-S.F.pol.tr.-S.F.sup.', 'R'), ('S..."
...,...,...,...
114,114,685,"[('S.Call.', 'L'), ('S.Call.-S.s.P.-S.intraCin..."
115,115,690,"[('S.Po.C.', 'L'), ('S.C.-S.Po.C.', 'L'), ('F...."
116,116,694,"[('S.T.i.-S.T.s.-S.T.pol.', 'L'), ('Lobule_par..."
117,117,699,"[('S.F.median-S.F.pol.tr.-S.F.sup.', 'R'), ('S..."


In [6]:
dir_path = "/neurospin/tmp/fred/models/2025-11-10"

In [7]:

def read_error_csv(path):

    if os.path.exists(path):
        df = pd.read_csv(path)
        return df
    return None

In [8]:

selected_index = np.load("/neurospin/dico/fred/Runs/01_betaVAE/Program/2023_jlaval_STSbabies/betaVAE/notebooks/fred/PEPR_Marseille/All the subjects/histogram_non_zero/index_to_save.npy")

In [9]:
def strip_dots(t):
    return tuple(s.replace(".", "") for s in t)

In [24]:
import ast

In [27]:

subfolders = sorted([
    f for f in os.listdir(dir_path)
    if os.path.isdir(os.path.join(dir_path, f))
])



# Table to  save the ks distance
df_entropy = pd.DataFrame(columns=["region", "p"])
df_chamfer = pd.DataFrame(columns=["region", "p"])
df_SVM = pd.DataFrame(columns=["region", "p"])


for i, folder in enumerate(subfolders):

    regions=re.match(r"(.*?(left|right))", folder).group(1)

    current_path = os.path.join(dir_path, folder)

    ce_file= os.path.join(current_path, "PEPR_Marseille", "Quantiles_Reconstruction_error.csv")
    chamfer_file= os.path.join(current_path, "PEPR_Marseille", "Quantiles_Chamfer_distance.csv")
    SVM_file= os.path.join(current_path, "PEPR_Marseille", "Quantiles_Residualisation_Score_SVM.csv")

    # lecture
    ce = read_error_csv(ce_file)
    chamfer = read_error_csv(chamfer_file)
    SVM = read_error_csv(SVM_file)

    ce= ce.loc[selected_index]
    chamfer= chamfer.loc[selected_index]
    SVM= SVM.loc[selected_index]

    if regions.endswith("_left"):
        regions_clear = regions.removesuffix("_left")
        region_search = (regions_clear, 'L')

    elif regions.endswith("_right"):
        regions_clear = regions.removesuffix("_right")
        region_search = (regions_clear, 'R')

    y_true=[]
    y_ce=[]
    y_chamfer=[]
    y_SVM=[]

    for j in df_regions["index_subject"]:
        idx = df_regions.index[df_regions["index_subject"] == j]
        L=df_regions.loc[df_regions["index_subject"] == j, "list of regions"].iloc[0]
        L = ast.literal_eval(L)
        if any(strip_dots(t) ==region_search for t in L):

            y_true.append(1)
        else:
            y_true.append(0)

        y_ce.append(ce.iloc[j,-1])
        y_chamfer.append(chamfer.iloc[j,-1])
        y_SVM.append(SVM.iloc[j,-1])
    print(np.sum(np.array(y_true)))
    df_entropy.loc[i] = [regions, roc_auc_score(y_true, y_ce)]
    df_chamfer.loc[i] = [regions, roc_auc_score(y_true, y_chamfer)]
    df_SVM.loc[i] = [regions, roc_auc_score(y_true, y_SVM)]



19
11
4
5
13
17
12
6
6
5
9
6
5
3
7
7
12
6
6
4
5
3
3
1
10
8
6
5
8
4
7
10
3
1
6
11
1
3
1
3
4
5
9
4
20
12
29
15
22
11
16
7
5
3
4
4


In [28]:
df_entropy.to_csv("entropy")
df_chamfer.to_csv("chamfer")
df_SVM.to_csv("SVM")
